In [23]:
import re
import matplotlib.pyplot as plt
from matplotlib.collections import EventCollection
import numpy as np

class pathdata(object):
    def __init__(self, sourcefile):
        self.source = sourcefile
        self.perc = {90: [], 98: []}
        self.percav = {}
        
        self.fill_data()

        
        
    def fill_data(self):
        with open(self.source, "r") as myfile:
            file_learned = True
            for line in myfile:
                is_valid = re.search("Target: \d+", line)
                if is_valid:
                    p = int(re.search("\d+", is_valid.group()).group())
                    err = float(re.search("\d+\.\d+", re.search("Err: \d+\.\d+", line).group()).group())
                    self.perc[p].append(err)
                if file_learned and re.search("nan", line):
                    file_learned = False
                    print("Error occurred in file {}".format(self.source))
        for p in self.perc.keys():
            self.percav[p] = np.nanmean(self.perc[p])
            
            
            
class summarydata_withstd(object):
    def __init__(self, files):
        self.files = files
        self.avgerror = 0
        self.avgerror_std = None
        self.avgsparsity = 0
        self.avgsparsity_std = None
        
        self.percs = {}
        self.percs_std = {}
        
        self.summary()
        
    def summary(self):
        pathdatadict = {}
        percs1 = {90: [], 98: []}
        for i in range(len(self.files)):
            pathdatadict[i] = pathdata(self.files[i])
            for p in percs1.keys():
                percs1[p].append(pathdatadict[i].percav[p])
            
        for p in percs1.keys():
            self.percs[p] = np.nanmean(percs1[p])
            self.percs_std[p] = np.nanstd(percs1[p])
            
            
class tablemaker(object):
    def __init__(self, filedict):
        self.filedict = filedict
        self.filenames = []
        self.ps = {90: [], 98: []}
        self.ps_std = {90: [], 98: []}
        
        self.makedata()
        
    def makedata(self):
        for f in self.filedict.keys():
            self.filenames.append(f)
            fsum = summarydata_withstd(self.filedict[f])
            for p in self.ps.keys():
                self.ps[p].append(fsum.percs[p])
                self.ps_std[p].append(fsum.percs_std[p])
                
    def maketable(self):
        print(f'\\begin{{tabular}}{{c|' + ''.join(['c' for i in range(len(self.filenames))]) + f'}} \\hline')
        print(' & ' + ' & '.join(self.filenames) + f' \\\\ \\hline')
        for p in self.ps.keys():
            print('{{\\bf {}\\% Sparse Err.}} & '.format(str(p)) + ' & '.join(['{:.2f} ({:.2f})'.format(self.ps[p][i], self.ps_std[p][i]) for i in range(len(self.ps[p]))]) + f' \\\\')
        print(f'\\end{{tabular}}')

In [24]:
testdict = { }
testdict['Torus'] = ['FullSparse/torusl1dog{}.txt'.format(k) for k in range(5)]
testdict['No reg.'] = ['FullSparse/noreg{}.txt'.format(k) for k in range(5)]
testdict['$L_1$'] = ['FullSparse/l1{}.txt'.format(k) for k in range(5)]
testdict['Perm. DoG'] = ['FullSparse/perml1dog{}.txt'.format(k) for k in range(5)]


test = tablemaker(testdict)
test.maketable()

\begin{tabular}{c|cccc} \hline
 & Torus & No reg. & $L_1$ & Perm. DoG \\ \hline
{\bf 90\% Sparse Err.} & 6.29 (1.17) & 78.64 (12.64) & 72.96 (20.30) & 9.66 (0.30) \\
{\bf 98\% Sparse Err.} & 95.87 (1.33) & 96.18 (0.89) & 95.77 (1.22) & 94.58 (1.60) \\
\end{tabular}


/tmp/ipykernel_3554897/1678024297.py:29: RuntimeWarning: Mean of empty slice
  self.percav[p] = np.nanmean(self.perc[p])


In [25]:
modulidict = { }
modulidict['Torus'] = ['FullSparse/torusl1dog{}.txt'.format(k) for k in range(5)]
modulidict['Circle'] = ['FullSparse/circlel1dog{}.txt'.format(k) for k in range(5)]
modulidict['Sphere'] = ['FullSparse/spherel1dog{}.txt'.format(k) for k in range(5)]
modulidict['Klein'] = ['FullSparse/kleinl1dog{}.txt'.format(k) for k in range(5)]
modulidict['6 Torus'] = ['FullSparse/torus6l1dog{}.txt'.format(k) for k in range(5)]


moduli = tablemaker(modulidict)
moduli.maketable()

\begin{tabular}{c|ccccc} \hline
 & Torus & Circle & Sphere & Klein & 6 Torus \\ \hline
{\bf 90\% Sparse Err.} & 6.29 (1.17) & 25.95 (33.92) & 69.12 (12.06) & 5.89 (0.58) & 14.52 (5.84) \\
{\bf 98\% Sparse Err.} & 95.87 (1.33) & 94.53 (0.78) & 87.75 (5.88) & 96.07 (0.65) & 94.96 (1.36) \\
\end{tabular}


/tmp/ipykernel_3554897/1678024297.py:29: RuntimeWarning: Mean of empty slice
  self.percav[p] = np.nanmean(self.perc[p])


In [26]:
inhibdict = {}
inhibdict['DoG'] = ['FullSparse/torusl1dog{}.txt'.format(k) for k in range(5)]
inhibdict['Diffusion'] = ['FullSparse/torusl1sq{}.txt'.format(k) for k in range(5)]
inhib = tablemaker(inhibdict)
inhib.maketable()

\begin{tabular}{c|ccc} \hline
 & DoG & Diffusion & Ripple \\ \hline
{\bf 90\% Sparse Err.} & 6.29 (1.17) & 24.19 (34.50) & 24.19 (34.50) \\
{\bf 98\% Sparse Err.} & 95.87 (1.33) & 94.55 (0.37) & 94.55 (0.37) \\
\end{tabular}


/tmp/ipykernel_3554897/1678024297.py:29: RuntimeWarning: Mean of empty slice
  self.percav[p] = np.nanmean(self.perc[p])


# Lottery

In [17]:
import re
import matplotlib.pyplot as plt
from matplotlib.collections import EventCollection
import numpy as np

class pathdata(object):
    def __init__(self, sourcefile):
        self.source = sourcefile
        self.perc = {"90\\% Sparse": [], "Lottery": []}
        self.percav = {}
        
        self.fill_data()

        
        
    def fill_data(self):
        with open(self.source, "r") as myfile:
            file_learned = True
            is_lot = False
            for line in myfile:
                is_valid = re.search("Validator: ", line)
                if not is_lot:
                    is_lot = re.search("Sparsity: 90", line)
                    if is_valid:
                        err = float(re.search("\d+\.\d+", re.search("Err: \d+\.\d+", line).group()).group())
                        self.perc["90\\% Sparse"].append(err)
                if is_lot: 
                    if is_valid:
                        err = float(re.search("\d+\.\d+", re.search("Err: \d+\.\d+", line).group()).group())
                        self.perc["Lottery"].append(err)
                if file_learned and re.search("nan", line):
                    file_learned = False
                    print("Error occurred in file {}".format(self.source))
        for p in self.perc.keys():
            self.percav[p] = np.nanmean(self.perc[p])
            
            
            
class summarydata_withstd(object):
    def __init__(self, files):
        self.files = files
        self.avgerror = 0
        self.avgerror_std = None
        self.avgsparsity = 0
        self.avgsparsity_std = None
        
        self.percs = {}
        self.percs_std = {}
        
        self.summary()
        
    def summary(self):
        pathdatadict = {}
        percs1 = {"90\\% Sparse": [], "Lottery": []}
        for i in range(len(self.files)):
            pathdatadict[i] = pathdata(self.files[i])
            for p in percs1.keys():
                percs1[p].append(pathdatadict[i].percav[p])
            
        for p in percs1.keys():
            self.percs[p] = np.nanmean(percs1[p])
            self.percs_std[p] = np.nanstd(percs1[p])
            
            
class tablemaker(object):
    def __init__(self, filedict):
        self.filedict = filedict
        self.filenames = []
        self.ps = {"90\\% Sparse": [], "Lottery": []}
        self.ps_std = {"90\\% Sparse": [], "Lottery": []}
        
        self.makedata()
        
    def makedata(self):
        for f in self.filedict.keys():
            self.filenames.append(f)
            fsum = summarydata_withstd(self.filedict[f])
            for p in self.ps.keys():
                self.ps[p].append(fsum.percs[p])
                self.ps_std[p].append(fsum.percs_std[p])
                
    def maketable(self):
        print(f'\\begin{{tabular}}{{c|' + ''.join(['c' for i in range(len(self.filenames))]) + f'}} \\hline')
        print(' & ' + ' & '.join(self.filenames) + f' \\\\ \\hline')
        for p in self.ps.keys():
            print('{{\\bf {}\\% Sparse Err.}} & '.format(str(p)) + ' & '.join(['{:.2f} ({:.2f})'.format(self.ps[p][i], self.ps_std[p][i]) for i in range(len(self.ps[p]))]) + f' \\\\')
        print(f'\\end{{tabular}}')

In [14]:
testdict = { }
testdict['Torus'] = ['Lottery/torusl1dog{}.txt'.format(k) for k in range(1, 2)]
testdict['No reg.'] = ['Lottery/noreg{}.txt'.format(k) for k in range(1, 2)]
testdict['$L_1$'] = ['Lottery/l1{}.txt'.format(k) for k in range(1, 2)]
testdict['Perm. DoG'] = ['Lottery/perml1dog{}.txt'.format(k) for k in range(1, 2)]


test = tablemaker(testdict)
test.maketable()

\begin{tabular}{c|cccc} \hline
 & Torus & No reg. & $L_1$ & Perm. DoG \\ \hline
{\bf 90\% Sparse\% Sparse Err.} & 5.37 (0.00) & 82.43 (0.00) & 74.83 (0.00) & 10.57 (0.00) \\
{\bf Lottery\% Sparse Err.} & 9.42 (0.00) & 68.83 (0.00) & 59.26 (0.00) & 34.97 (0.00) \\
\end{tabular}


In [15]:
modulidict = { }
modulidict['Torus'] = ['Lottery/torusl1dog{}.txt'.format(k) for k in range(1, 2)]
modulidict['Circle'] = ['Lottery/circlel1dog{}.txt'.format(k) for k in range(1, 2)]
modulidict['Sphere'] = ['Lottery/spherel1dog{}.txt'.format(k) for k in range(1, 2)]
modulidict['Klein'] = ['Lottery/kleinl1dog{}.txt'.format(k) for k in range(1, 2)]
modulidict['6 Torus'] = ['Lottery/torus6l1dog{}.txt'.format(k) for k in range(1, 2)]


moduli = tablemaker(modulidict)
moduli.maketable()

\begin{tabular}{c|ccccc} \hline
 & Torus & Circle & Sphere & Klein & 6 Torus \\ \hline
{\bf 90\% Sparse\% Sparse Err.} & 5.37 (0.00) & 9.60 (0.00) & 23.85 (0.00) & 5.48 (0.00) & 38.12 (0.00) \\
{\bf Lottery\% Sparse Err.} & 9.42 (0.00) & 96.18 (0.00) & 94.19 (0.00) & 9.74 (0.00) & 94.22 (0.00) \\
\end{tabular}


In [19]:
inhibdict = {}
inhibdict['DoG'] = ['Lottery/torusl1dog{}.txt'.format(k) for k in range(1, 2)]
inhibdict['Square'] = ['Lottery/torusl1sq{}.txt'.format(k) for k in range(1, 2)]
inhib = tablemaker(inhibdict)
inhib.maketable()

\begin{tabular}{c|cc} \hline
 & DoG & Square \\ \hline
{\bf 90\% Sparse\% Sparse Err.} & 5.37 (0.00) & 5.82 (0.00) \\
{\bf Lottery\% Sparse Err.} & 9.42 (0.00) & 10.74 (0.00) \\
\end{tabular}
